# Extract aperiodic component of Sleep Data 

This script loads the pre-processed polysomnography data and extracts the apeiodic component of the EEG signal using the fooof algorithm (https://github.com/fooof-tools/fooof) and the IRASA algorithm (https://doi.org/10.1007/s10548-015-0448-0). The aperiodic component is then saved as a .csv file along with other spectrally derived information and specifically utilized parameters. 

In [2]:
%matplotlib inline

## Import packages 
import yasa
import pickle
from fooof import FOOOF
from fooof.objs.utils import average_fg, combine_fooofs, compare_info
from fooof.bands import Bands
import numpy as np
import os
import mne
from tqdm import tqdm
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import scipy.signal as signal 
import scipy.stats as stats
mne.set_log_level('WARNING')

# Set plotting settings
sns.set_context('talk')
sns.set_style('white')
sns.set_palette('colorblind')

In [3]:
## Adjusted YASA functions for computing aperiodic power spectrum based on IRASA method

def foi_range_irasa(foi=(1,45), hset_max=1.9):
    """

    Calculate the frequency range to evaluate the IRASA power spectrum.

    Parameters
    ----------
    foi : tuple of int or float
        Frequency range of interest.
    hset_max : float
        Maximum resampling factor.

    """

    freq_eval_min = foi[0] / hset_max
    freq_eval_max = foi[1] * hset_max

    return freq_eval_min, freq_eval_max

def irasa(data, sf=None, ch_names=None, band=(1, 45),
          hset=[1.1,1.15,1.2,1.25,1.3,1.35,1.4,1.45,1.5,
                1.55,1.6,1.65,1.7,1.75,1.8,1.85,1.9],
          return_fit=True,
          win_sec=4,
          kwargs_welch=dict(average="median", window="hamming"),
          verbose=True,
          ):
    """
    Separate the aperiodic (= fractal, or 1/f) and oscillatory component
    of the power spectra of EEG data using the IRASA method.

    .. versionadded:: 0.1.7

    Parameters
    ----------
    data : :py:class:`numpy.ndarray` or :py:class:`mne.io.BaseRaw`
        1D or 2D EEG data. Can also be a :py:class:`mne.io.BaseRaw`, in which
        case ``data``, ``sf``, and ``ch_names`` will be automatically
        extracted, and ``data`` will also be converted from Volts (MNE default)
        to micro-Volts (YASA).
    sf : float
        The sampling frequency of data AND the hypnogram.
        Can be omitted if ``data`` is a :py:class:`mne.io.BaseRaw`.
    ch_names : list
        List of channel names, e.g. ['Cz', 'F3', 'F4', ...]. If None,
        channels will be labelled ['CHAN000', 'CHAN001', ...].
        Can be omitted if ``data`` is a :py:class:`mne.io.BaseRaw`.
    band : tuple or None
        Broad band frequency range.
        Default is 1 to 30 Hz.
    hset : list or :py:class:`numpy.ndarray`
        Resampling factors used in IRASA calculation. Default is to use a range
        of values from 1.1 to 1.9 with an increment of 0.05.
    return_fit : boolean
        If True (default), fit an exponential function to the aperiodic PSD
        and return the fit parameters (intercept, slope) and :math:`R^2` of
        the fit.

        The aperiodic signal, :math:`L`, is modeled using an exponential
        function in semilog-power space (linear frequencies and log PSD) as:

        .. math:: L = a + \text{log}(F^b)

        where :math:`a` is the intercept, :math:`b` is the slope, and
        :math:`F` the vector of input frequencies.
    win_sec : int or float
        The length of the sliding window, in seconds, used for the Welch PSD
        calculation. Ideally, this should be at least two times the inverse of
        the lower frequency of interest (e.g. for a lower frequency of interest
        of 0.5 Hz, the window length should be at least 2 * 1 / 0.5 =
        4 seconds).
    kwargs_welch : dict
        Optional keywords arguments that are passed to the
        :py:func:`scipy.signal.welch` function.
    verbose : bool or str
        Verbose level. Default (False) will only print warning and error
        messages. The logging levels are 'debug', 'info', 'warning', 'error',
        and 'critical'. For most users the choice is between 'info'
        (or ``verbose=True``) and warning (``verbose=False``).

    Returns
    -------
    freqs : :py:class:`numpy.ndarray`
        Frequency vector.
    psd_aperiodic : :py:class:`numpy.ndarray`
        The fractal (= aperiodic) component of the PSD.
    psd_oscillatory : :py:class:`numpy.ndarray`
        The oscillatory (= periodic) component of the PSD.
    fit_params : :py:class:`pandas.DataFrame` (optional)
        Dataframe of fit parameters. Only if ``return_fit=True``.

    Notes
    -----
    The Irregular-Resampling Auto-Spectral Analysis (IRASA) method is
    described in Wen & Liu (2016). In a nutshell, the goal is to separate the
    fractal and oscillatory components in the power spectrum of EEG signals.

    The steps are:

    1. Compute the original power spectral density (PSD) using Welch's method.
    2. Resample the EEG data by multiple non-integer factors and their
       reciprocals (:math:`h` and :math:`1/h`).
    3. For every pair of resampled signals, calculate the PSD and take the
       geometric mean of both. In the resulting PSD, the power associated with
       the oscillatory component is redistributed away from its original
       (fundamental and harmonic) frequencies by a frequency offset that varies
       with the resampling factor, whereas the power solely attributed to the
       fractal component remains the same power-law statistical distribution
       independent of the resampling factor.
    4. It follows that taking the median of the PSD of the variously
       resampled signals can extract the power spectrum of the fractal
       component, and the difference between the original power spectrum and
       the extracted fractal spectrum offers an approximate estimate of the
       power spectrum of the oscillatory component.

    Note that an estimate of the original PSD can be calculated by simply
    adding ``psd = psd_aperiodic + psd_oscillatory``.

    For an example of how to use this function, please refer to
    https://github.com/raphaelvallat/yasa/blob/master/notebooks/09_IRASA.ipynb

    For an article discussing the challenges of using IRASA (or fooof) see [5].

    References
    ----------
    [1] Wen, H., & Liu, Z. (2016). Separating Fractal and Oscillatory
        Components in the Power Spectrum of Neurophysiological Signal.
        Brain Topography, 29(1), 13–26.
        https://doi.org/10.1007/s10548-015-0448-0

    [2] https://github.com/fieldtrip/fieldtrip/blob/master/specest/

    [3] https://github.com/fooof-tools/fooof

    [4] https://www.biorxiv.org/content/10.1101/299859v1

    [5] https://doi.org/10.1101/2021.10.15.464483
    """
    import fractions
    from yasa.io import set_log_level
    import logging

    set_log_level(verbose)
    # Check if input data is a MNE Raw object
    if isinstance(data, mne.io.BaseRaw):
        sf = data.info["sfreq"]  # Extract sampling frequency
        ch_names = data.ch_names  # Extract channel names
        hp = data.info["highpass"]  # Extract highpass filter
        lp = data.info["lowpass"]  # Extract lowpass filter
        data = data.get_data(units=dict(eeg="uV", emg="uV", eog="uV", ecg="uV"))
    else:
        # Safety checks
        assert isinstance(data, np.ndarray), "Data must be a numpy array."
        data = np.atleast_2d(data)
        assert data.ndim == 2, "Data must be of shape (nchan, n_samples)."
        nchan, npts = data.shape
        assert nchan < npts, "Data must be of shape (nchan, n_samples)."
        assert sf is not None, "sf must be specified if passing a numpy array."
        assert isinstance(sf, (int, float))
        if ch_names is None:
            ch_names = ["CHAN" + str(i).zfill(3) for i in range(nchan)]
        else:
            ch_names = np.atleast_1d(np.asarray(ch_names, dtype=str))
            assert ch_names.ndim == 1, "ch_names must be 1D."
            assert len(ch_names) == nchan, "ch_names must match data.shape[0]."
        hp = 0  # Highpass filter unknown -> set to 0 Hz
        lp = sf / 2  # Lowpass filter unknown -> set to Nyquist

    # Check the other arguments
    hset = np.asarray(hset)
    assert hset.ndim == 1, "hset must be 1D."
    assert hset.size > 1, "2 or more resampling fators are required."
    hset = np.round(hset, 4)  # avoid float precision error with np.arange.
    band = sorted(band)
    assert band[0] > 0, "first element of band must be > 0."
    assert band[1] < (sf / 2), "second element of band must be < (sf / 2)."
    win = int(win_sec * sf)  # nperseg

    # Inform about maximum resampled fitting range
    h_max = np.max(hset)
    band_evaluated = (band[0] / h_max, band[1] * h_max)
    freq_Nyq = sf / 2  # Nyquist frequency
    freq_Nyq_res = freq_Nyq / h_max  # minimum resampled Nyquist frequency
    logging.info(f"Fitting range: {band[0]:.2f}Hz-{band[1]:.2f}Hz")
    logging.info(f"Evaluated frequency range: {band_evaluated[0]:.2f}Hz-{band_evaluated[1]:.2f}Hz")
    if band_evaluated[0] < hp:
        logging.warning(
            "The evaluated frequency range starts below the "
            f"highpass filter ({hp:.2f}Hz). Increase the lower band"
            f" ({band[0]:.2f}Hz) or decrease the maximum value of "
            f"the hset ({h_max:.2f})."
        )
    if band_evaluated[1] > lp and lp < freq_Nyq_res:
        logging.warning(
            "The evaluated frequency range ends after the "
            f"lowpass filter ({lp:.2f}Hz). Decrease the upper band"
            f" ({band[1]:.2f}Hz) or decrease the maximum value of "
            f"the hset ({h_max:.2f})."
        )
    if band_evaluated[1] > freq_Nyq_res:
        logging.warning(
            "The evaluated frequency range ends after the "
            "resampled Nyquist frequency "
            f"({freq_Nyq_res:.2f}Hz). Decrease the upper band "
            f"({band[1]:.2f}Hz) or decrease the maximum value "
            f"of the hset ({h_max:.2f})."
        )

    # Calculate the original PSD over the whole data
    freqs, psd = signal.welch(data, sf, nperseg=win, **kwargs_welch)

    # Start the IRASA procedure
    psds = np.zeros((len(hset), *psd.shape))

    for i, h in enumerate(hset):
        # Get the upsampling/downsampling (h, 1/h) factors as integer
        rat = fractions.Fraction(str(h))
        up, down = rat.numerator, rat.denominator
        # Much faster than FFT-based resampling
        data_up = signal.resample_poly(data, up, down, axis=-1)
        data_down = signal.resample_poly(data, down, up, axis=-1)
        # Calculate the PSD using same params as original
        freqs_up, psd_up = signal.welch(data_up, h * sf, nperseg=win, **kwargs_welch)
        freqs_dw, psd_dw = signal.welch(data_down, sf / h, nperseg=win, **kwargs_welch)
        # Geometric mean of h and 1/h
        psds[i, :] = np.sqrt(psd_up * psd_dw)

    # Now we take the median PSD of all the resampling factors, which gives
    # a good estimate of the aperiodic component of the PSD.
    psd_aperiodic = np.median(psds, axis=0)

    # We can now calculate the oscillations (= periodic) component.
    psd_osc = psd - psd_aperiodic

    # Let's crop to the frequencies defined in band
    mask_freqs = np.ma.masked_outside(freqs, *band).mask
    freqs = freqs[~mask_freqs]
    psd_aperiodic = np.compress(~mask_freqs, psd_aperiodic, axis=-1)
    psd_osc = np.compress(~mask_freqs, psd_osc, axis=-1)

    if return_fit:
        # Aperiodic fit in semilog space for each channel
        from scipy.optimize import curve_fit

        intercepts, slopes, r_squared = [], [], []

        def func(t, a, b):
            # See https://github.com/fooof-tools/fooof
            return a + np.log10(t**b)

        for y in np.atleast_2d(psd_aperiodic):
            y_log = np.log10(y)
            # Note that here we define bounds for the slope but not for the
            # intercept.
            popt, pcov = curve_fit(
                func, freqs, y_log, p0=(2, -1), bounds=((-np.inf, -10), (np.inf, 2))
            )
            intercepts.append(popt[0])
            slopes.append(popt[1])
            # Calculate R^2: https://stackoverflow.com/q/19189362/10581531
            residuals = y_log - func(freqs, *popt)
            ss_res = np.sum(residuals**2)
            ss_tot = np.sum((y_log - np.mean(y_log)) ** 2)
            r_squared.append(1 - (ss_res / ss_tot))

        # Create fit parameters dataframe
        fit_params = {
            "Chan": ch_names,
            "Intercept": intercepts,
            "Slope": slopes,
            "R^2": r_squared,
            "std(osc)": np.std(psd_osc, axis=-1, ddof=1),
        }
        return freqs, psd_aperiodic, psd_osc, pd.DataFrame(fit_params)
    else:
        return freqs, psd_aperiodic, psd_osc

def compute_irasa_power(data, hypnogram, sf, subject, band=(1, 45), h_set_max=1.5):
    """
    Compute the IRASA power spectrum for each sleep stage.

    Parameters:
    -----------
    data : ndarray
        2D EEG data where columns represent time points and rows represent different channels.
    hypnogram : ndarray
        1D array representing the sleep stage for each time point in data.
    sf : float
        Sampling frequency of the EEG data.
    subject : str
        Identifier for the subject whose data is being processed.
    band : tuple
        Frequency band to compute the IRASA power spectrum over.

    Returns:
    --------
    fit_df : DataFrame
        A DataFrame containing fit parameters for each sleep stage.
    psd_df : DataFrame
        A DataFrame containing power spectral density values for each sleep stage.

    Notes:
    ------
    This function uses the YASA library's IRASA method to separate oscillatory and aperiodic components 
    of the EEG power spectrum for each sleep stage. It then computes bandpower for each component and optionally
    plots the PSD for each component.
    """

    fit_dfs = []
    psd_aperiodic_dict = {}
    psd_oscillatory_dict = {}
    
    for stage in np.unique(hypnogram):
        if stage >= 0:
            stage_data = data[:, hypnogram == int(stage)]

            # Extract aperiodic and oscillatory components of the PSD
            freqs, psd_aperiodic, psd_oscillatory, fit_params = irasa( # type: ignore
                stage_data, sf=sf, ch_names=['C3', 'C4'], band=band,
                return_fit=True, win_sec=2,
                hset=np.arange(1.0, h_set_max + 0.05, 0.05),
                kwargs_welch=dict(average='median', window='hamming')
            )
            fit_params['Stage'] = stage
            fit_params['Subject'] = subject
            fit_dfs.append(fit_params)

            for idx, channel in enumerate(['C3', 'C4']):
                key = (stage, channel)
                psd_aperiodic_dict[key] = psd_aperiodic[idx, :]
                psd_oscillatory_dict[key] = psd_oscillatory[idx, :]
            
    fit_df = pd.concat(fit_dfs, axis=0)

    return freqs, psd_aperiodic_dict, psd_oscillatory_dict, fit_df.reset_index(drop=True)

## Plotting function for spectral components for both IRASA and FOOOF

def plot_spectral_components(freqs=None, psd_osc=None, psd_ap=None, fm=None, chan=None, 
                             stage=None, fit_params=None, log_freq=True, method='IRASA',
                             save=False, fig_path=None):
    if method == 'FOOOF':
        # Plot FOOOF results
        fig = plt.figure() 
        plt.tight_layout()
        plt.yscale('log')
        if log_freq:
            plt.xscale('log')
        plt.plot(fm.freqs, 10**(fm.power_spectrum), c='k', label="Power spectrum", lw=2)
        plt.plot(fm.freqs, 10**(fm._ap_fit), c='b',linestyle='--', label="Aperiodic fit", lw=2)
        plt.plot(fm.freqs, 10**(fm.fooofed_spectrum_), c='r', label="FOOOF model fit", lw=2)
        fig.suptitle('Full signal PSD with FOOOF result')
        fig.axes[0].set_xlabel("Frequency (Hz)")
        fig.axes[0].set_ylabel("PSD log($V^2$/Hz)")
        fig.axes[0].legend()
        # set text with fit parameters
        fig.axes[0].text(0.1, 0.5, f"Slope: {round(fm.aperiodic_params_[1], 2)}", 
                         transform=fig.axes[0].transAxes)
        sns.despine()

    elif method == 'IRASA':
        # Plot IRASA results
        fig = plt.figure() 
        plt.tight_layout()
        plt.yscale('log')
        if log_freq:
            plt.xscale('log')
        plt.plot(freqs, (psd_ap+psd_osc).ravel(), c='k', label="Power spectrum", lw=2)
        plt.plot(freqs, psd_ap.ravel(), c='b',linestyle='--', label="Aperiodic fit", lw=2)
        #plt.plot(freqs, psd_osc.ravel(), c='cyan',linestyle='--', label="Periodic components", lw=2)
        fig.suptitle('Full signal PSD with IRASA result')
        fig.axes[0].set_xlabel("Frequency (Hz)")
        fig.axes[0].set_ylabel("PSD log($V^2$/Hz)")
        fig.axes[0].legend()
        # set text with fit parameters 
        slope = fit_params.set_index(['Chan','Stage']).loc[chan, stage]['Slope']
        fig.axes[0].text(0.1, 0.5, f"Slope: {round(slope,2)}",
                         transform=fig.axes[0].transAxes)    
        sns.despine()   

    if save:
        plt.savefig(f'{fig_path}.png')
        plt.close('all')

    return fig

## Function to compute the aperiodic exponent with FOOOF for each sleep stage for channels C3 and C4

def compute_fooof_aperiodic_exponent(data, sf, hypnogram, subject, band=(1, 45)):
    aperiodic_exponent_dfs = []
    fg_dict = {}

    # Compute the power spectrum for each sleep stage
    for stage in np.unique(hypnogram):
        if stage >= 0:
            # Slice the data based on sleep stage
            stage_data = data[:, hypnogram == stage]
            # Slice the data based on channel
            for idx, ch_name in enumerate(['C3', 'C4']):
                # Initialize FOOOF object for each channel-stage combination
                fg = FOOOF(max_n_peaks=5, aperiodic_mode='fixed', peak_width_limits=(0.5, 12.0), 
                           min_peak_height=0.0, peak_threshold=2.0, verbose=False)
                
                # Compute the power spectrum
                freqs, psd = signal.welch(stage_data[idx, :], sf, nperseg=int(2*sf), 
                                          average='median', window='hamming')
                freqs_foi = freqs[(freqs >= band[0]) & (freqs <= band[1])]
                psd_foi = psd[(freqs >= band[0]) & (freqs <= band[1])]
                
                # Fit the FOOOF model
                fg.fit(freqs, psd, freq_range=band)
                
                # Store the FOOOF object to dictionary
                key = (stage, ch_name)
                fg_dict[key] = fg
                
                # Extract the aperiodic exponent
                aperiodic_exponent = fg.get_params('aperiodic_params', 'exponent')
                
                # Extract the R^2 value
                r2s = fg.get_params('r_squared')
                
                # Extract the intercept
                intercept = fg.get_params('aperiodic_params', 'offset')
                
                # Extract the standard deviation of the oscillatory component
                std_osc = np.std(fg.power_spectrum - fg._ap_fit, ddof=1, axis=-1)
                
                # Store the aperiodic exponent
                aperiodic_exponent_df = pd.DataFrame({
                    'Subject': [subject]*len(freqs_foi),
                    'Stage': [stage]*len(freqs_foi),
                    'Chan': [ch_name]*len(freqs_foi),
                    'Slope': [aperiodic_exponent]*len(freqs_foi),
                    'R^2': [r2s]*len(freqs_foi), 
                    'Intercept': [intercept]*len(freqs_foi),
                    'std(osc)': [std_osc]*len(freqs_foi),
                    'Spectrum': psd_foi,
                    'Frequency': freqs_foi
                })
                aperiodic_exponent_dfs.append(aperiodic_exponent_df)

    # Combine all dataframes
    aperiodic_exponent_df_all = pd.concat(aperiodic_exponent_dfs, axis=0)

    return fg_dict, aperiodic_exponent_df_all.reset_index(drop=True)
    

## 1. Load data

In [4]:
# Set paths
path = '/mnt/server/data03/2023_NENA_Aperiodic_Workshop/data/processed/'
fig_path = '/mnt/server/data03/2023_NENA_Aperiodic_Workshop/figures/subject/'
# Obtain list of unique recordings
files = list(set(["-".join(f.split('-')[0:3]) for f in os.listdir(path)]))

## 2. Process and extract aperiodic component from recording files

In [4]:
## Iterate over all files and process them
fit_dfs, fooof_dfs, psd_aperiodic_all, psd_oscillatory_all, fooof_groups_all = [], [], [], [], []
for idx, file in enumerate(tqdm(files)):
    print(f'Extracting aperiodic components from file : {file}')
    # Load the data and hypnogram files
    subject = file.split('-')[-1]
    raw = mne.io.read_raw_fif(path + file + '-raw.fif.gz', preload=True)
    hypnogram = np.load(path + file + '-hypnogram_with_art.npy')
    # Get sampling frequency
    sf = raw.info['sfreq']
    # Get data
    data = raw.get_data(['C3','C4'], units='uV') 
    # Extract aperiodic component with IRASA from yasa per sleep stage
    freqs, psd_aperiodic, psd_oscillatory, fit_df = compute_irasa_power(data, hypnogram, sf, subject, band=(1, 45), h_set_max=1.5)
    fit_df['Method'] = 'IRASA'
    fit_dfs.append(fit_df)
    psd_aperiodic_all.append(psd_aperiodic)
    psd_oscillatory_all.append(psd_oscillatory)
    # Extract aperiodic component with FOOOF per sleep stage
    fg, fooof_df = compute_fooof_aperiodic_exponent(data, sf, hypnogram, subject, band=(1, 45))
    fooof_df['Method'] = 'FOOOF'   
    fooof_dfs.append(fooof_df)
    fooof_groups_all.append(fg)
    # Plot spectral results with IRASA and FOOOF
    for chan in ['C3', 'C4']:
        for stage in [0, 1, 2, 3, 4]:
            save_path = fig_path + f'{subject}_{stage}_{chan}'
            try:
                fig = plot_spectral_components(fm=fg[(stage, chan)], log_freq=True, method='FOOOF', 
                                               save=True, fig_path=save_path + '_FOOOF')
                fig2 = plot_spectral_components(freqs=freqs, psd_osc=psd_oscillatory[(stage, chan)], fit_params=fit_df,
                                                psd_ap=psd_aperiodic[(stage, chan)], log_freq=True, chan=chan, stage=stage,
                                                method='IRASA', save=True, fig_path=save_path + '_IRASA')
            except:
                print(f'No stage {stage} for {chan} for {subject}')
                pass

# Combine all dataframes
fit_df_all = pd.concat(fit_dfs, axis=0)
fooof_df_all = pd.concat(fooof_dfs, axis=0)
#psd_aperiodic_all
#psd_oscillatory_all
#fooof_groups_all

  0%|          | 0/59 [00:00<?, ?it/s]

Extracting aperiodic components from file : cfs-visit5-801225


  2%|▏         | 1/59 [01:29<1:26:05, 89.05s/it]

Extracting aperiodic components from file : cfs-visit5-801662


  3%|▎         | 2/59 [02:40<1:14:50, 78.78s/it]

...

Extracting aperiodic components from file : cfs-visit5-800249


 98%|█████████▊| 58/59 [1:15:30<01:18, 78.96s/it]

Extracting aperiodic components from file : cfs-visit5-801873


100%|██████████| 59/59 [1:16:54<00:00, 78.21s/it]


[{(0.0, 'C3'): <fooof.objs.fit.FOOOF at 0x7f5a48094f40>,
  (0.0, 'C4'): <fooof.objs.fit.FOOOF at 0x7f5a48096620>,
  (1.0, 'C3'): <fooof.objs.fit.FOOOF at 0x7f5a480979d0>,
  (1.0, 'C4'): <fooof.objs.fit.FOOOF at 0x7f5a480969b0>,
  (2.0, 'C3'): <fooof.objs.fit.FOOOF at 0x7f5a47ad46d0>,
  (2.0, 'C4'): <fooof.objs.fit.FOOOF at 0x7f5a47ad5cc0>,
  (3.0, 'C3'): <fooof.objs.fit.FOOOF at 0x7f5a47ad7220>,
  (3.0, 'C4'): <fooof.objs.fit.FOOOF at 0x7f5a47ad5c60>,
  (4.0, 'C3'): <fooof.objs.fit.FOOOF at 0x7f5a47ad44f0>,
  (4.0, 'C4'): <fooof.objs.fit.FOOOF at 0x7f5a47ad6500>},
 {(0.0, 'C3'): <fooof.objs.fit.FOOOF at 0x7f5a47e8be20>,
  (0.0, 'C4'): <fooof.objs.fit.FOOOF at 0x7f5a48094160>,
  (1.0, 'C3'): <fooof.objs.fit.FOOOF at 0x7f5a48095810>,
  (1.0, 'C4'): <fooof.objs.fit.FOOOF at 0x7f5a48094040>,
  (2.0, 'C3'): <fooof.objs.fit.FOOOF at 0x7f5a480959c0>,
  (2.0, 'C4'): <fooof.objs.fit.FOOOF at 0x7f5a48094c70>,
  (3.0, 'C3'): <fooof.objs.fit.FOOOF at 0x7f5a48095690>,
  (3.0, 'C4'): <fooof.objs.fit

In [5]:
# Merge fit_df_all and fooof_df_all on common columns
fit_dfs_all = pd.concat([fit_df_all, 
                         fooof_df_all.groupby(['Subject', 'Stage', 'Chan', 'Method']).mean().reset_index()], join='inner', axis=0)
fit_dfs_all['Slope'] = np.abs(fit_dfs_all['Slope'])
fit_dfs_all.reset_index(drop=True, inplace=True)
fit_dfs_all

,Chan,Intercept,Slope,R^2,std(osc),Stage,Subject,Method
0,C3,1.930738,1.680066,0.885381,0.624027,0.0,801225,IRASA
1,C4,1.842349,1.492382,0.872890,0.874594,0.0,801225,IRASA
2,C3,1.967682,2.035502,0.924771,0.328819,1.0,801225,IRASA
3,C4,1.914756,1.926399,0.924603,0.318299,1.0,801225,IRASA
4,C3,2.512133,2.553808,0.942534,0.420542,2.0,801225,IRASA
...,...,...,...,...,...,...,...,...
1163,C4,1.905306,2.156738,0.997980,0.135930,2.0,802739,FOOOF
1164,C3,2.557011,2.529979,0.989005,0.098169,3.0,802739,FOOOF
1165,C4,2.502161,2.600625,0.993477,0.128468,3.0,802739,FOOOF
1166,C3,1.558968,1.881564,0.996950,0.183944,4.0,802739,FOOOF


In [6]:
fit_dfs_all.to_csv('/mnt/server/data03/2023_NENA_Aperiodic_Workshop/results/aperiodic.csv', index=False)

In [7]:
# save psd and fooof dicts
with open('/mnt/server/data03/2023_NENA_Aperiodic_Workshop/results/psd_aperiodic_dict.pkl', 'wb') as f:
    pickle.dump(psd_aperiodic_all, f)
with open('/mnt/server/data03/2023_NENA_Aperiodic_Workshop/results/psd_oscillatory_dict.pkl', 'wb') as f:
    pickle.dump(psd_oscillatory_all, f)
with open('/mnt/server/data03/2023_NENA_Aperiodic_Workshop/results/fooof_groups_dict.pkl', 'wb') as f:
    pickle.dump(fooof_groups_all, f)